### Import Libary

In [1]:
import torch
import os
import yaml
import pandas as pd
import re

### Đọc danh sách file dữ liệu

In [2]:

dir_path = './data'
files_list = os.listdir(dir_path + os.sep)
files_list


['data.yml']

### Đọc dữ liệu câu hỏi và câu trả lời, lưu thành 2 list

In [3]:
questions = list()
answers = list()
categories = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    category = docs['categories'][0]

    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )
answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append(answers_with_tags[i] )


### Biến đổi các từ thành các cụm từ có nghĩa 

In [4]:
from pyvi import ViTokenizer

#! Đọc các cụm từ cần gộp từ file
def load_phrases(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        phrases = [line.strip() for line in file.readlines()]
    return phrases

file_path = 'phrases.txt'
custom_phrases = load_phrases(file_path)

def custom_tokenize(text):
    for phrase in custom_phrases:
        #! Gom nhóm các cụm từ có nghĩa
        text = text.replace(phrase, phrase.replace(" ", "_"))
    return ViTokenizer.tokenize(text)


def tokenize_sentences(sentences):
    result = []
    for sentence in sentences:
        result.append(custom_tokenize(sentence))
    return result


### Loại bỏ các ký tự đặc biệt và biến đổi thành chữ thường

In [5]:
def clean_text(text):
    # Loại bỏ khoảng trống dư thừa giữa các từ
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
    text = re.sub(r'\!', '', text) # xoa dau cham than   
    text = re.sub(r'\?', '', text) # xoa dau hoi cham
    text = re.sub(r'\.', '', text) # xoa dau cham cau
    text = re.sub(r'#([^\s]+)', r'\1',text) # xóa các hastag #
    text =  re.sub(r'["\'()[]{}]', '', text) # xoa cac dau ngoac kep vs ngoac don
    text = text.lower()
    return text.strip()

df = pd.DataFrame({
    'questions': questions,
    'answers': answers,
})

df['questions_processed'] = df['questions'].apply(clean_text)
df['questions_processed'] = tokenize_sentences(df['questions_processed'])

df.to_csv('data.csv',index=False)
print(df.shape)


(195, 3)


### Load model và Tokenizer của phoBert

In [6]:
from transformers import AutoModel, AutoTokenizer

PhobertTokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


### Embedding

In [7]:
sentences = df['questions_processed'].tolist()
inputs = PhobertTokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

print(embeddings)
print(embeddings.shape)


tensor([[ 0.0387,  0.1673,  0.1168,  ..., -0.0950, -0.1295,  0.2398],
        [ 0.2258, -0.1711,  0.2621,  ..., -0.0998, -0.3288,  0.1852],
        [ 0.2428,  0.1397,  0.0877,  ..., -0.0598, -0.1974,  0.2529],
        ...,
        [ 0.0876,  0.0446, -0.1104,  ..., -0.0713, -0.3734,  0.1000],
        [-0.0841,  0.0192,  0.0404,  ...,  0.1642, -0.1514, -0.0837],
        [-0.0035, -0.0552, -0.1650,  ...,  0.1581, -0.2096, -0.0708]])
torch.Size([195, 768])


In [8]:
embedding_list = embeddings.tolist()
df['embedding'] = embedding_list



In [9]:
import numpy as np

# Giả sử tensor_embeddings là tensor cần lưu
tensor_embeddings = embeddings.numpy()  # Chuyển từ Torch tensor sang NumPy array


np.save("embeddings.npy", tensor_embeddings)